The goal of this notebook is to understand the TensorFlow's Data API, some part of the TFRecord format and an introduction to TensorFlow Extended (TFX) for end-to-end ML pipeline. 

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [ ]:
!pip install -U tfx

RESTART THE COLAB RUNTIME WHEN YOU GET TO THIS CELL

In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.9.1
TFX version: 1.9.0


In [2]:
import tensorflow as tf

## TensorFlow Data API

In [3]:
list = range(10)
list

range(0, 10)

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(range(10))
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
for item in dataset:
    print(item.numpy())

0
1
2
3
4
5
6
7
8
9


In [6]:
it = iter(dataset)
next(it)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [7]:
next(it).numpy()

1

In [8]:
print(dataset.reduce(0, lambda state, value: state + value).numpy())

45


In [9]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item.numpy())

[0 1 2 3 4 5 6]
[7 8 9 0 1 2 3]
[4 5 6 7 8 9 0]
[1 2 3 4 5 6 7]
[8 9]


In [10]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
dataset = dataset.repeat(3)
for item in dataset:
    print(item.numpy())

1
2
3
4
5
1
2
3
4
5
1
2
3
4
5


In [11]:
dataset = dataset.map(lambda x: x**2)
for item in dataset:
    print(item.numpy())

1
4
9
16
25
1
4
9
16
25
1
4
9
16
25


In [12]:
del list

In [13]:
elements = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
dataset = tf.data.Dataset.from_generator(lambda: elements, tf.int64)

for batch in dataset:
    print(batch.numpy())

[1 2 3]
[4 5 6]
[7 8 9]


In [14]:
dataset = dataset.unbatch()
list(dataset.as_numpy_iterator())

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
for item in dataset:
    print(item.numpy())

1
2
3
4
5
6
7
8
9


In [16]:
dataset.element_spec

TensorSpec(shape=<unknown>, dtype=tf.int64, name=None)

In [17]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10], minval=1, maxval=10, dtype=tf.int32))
dataset1

<TensorSliceDataset element_spec=TensorSpec(shape=(10,), dtype=tf.int32, name=None)>

In [18]:
for item in dataset1:
    print(item.numpy())

[2 3 5 3 5 5 6 1 6 3]
[1 4 9 7 6 1 3 1 4 9]
[4 9 2 9 3 2 3 7 5 6]
[9 9 8 9 9 8 2 5 7 2]


In [19]:
dataset2 = tf.data.Dataset.from_tensor_slices((tf.random.uniform([4]), tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))
for item in dataset2:
    print(item)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.888528>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 5,  1,  3, 18, 43, 36, 84, 10,  8, 80, 28, 32, 72, 39, 51, 38, 76,
       14, 44, 32, 20, 35, 67, 18, 13, 70, 63, 80, 78, 49, 63, 87, 50, 51,
       52, 76,  6, 42, 78,  6,  7, 72,  2, 22, 20,  5, 50, 75, 81, 47, 75,
       27, 43, 11, 58,  4, 96, 43, 93, 73,  7, 12, 60, 83, 24,  0, 91, 44,
       26, 64, 47, 87, 46, 52, 31, 93, 80, 44, 59, 53, 88, 79,  8, 37, 39,
       43,  7, 44, 59, 86, 31,  4, 58, 17, 99, 32, 68, 67, 41,  5],
      dtype=int32)>)
(<tf.Tensor: shape=(), dtype=float32, numpy=0.07443571>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([14, 29, 23, 81, 27, 73,  2, 36, 76, 84, 27, 57, 50, 57, 79, 76, 45,
       51, 97, 25, 39, 85, 50, 31, 39,  1, 11, 19, 59, 92, 51, 68, 54, 30,
       69, 36, 18, 77, 85, 19, 41, 85, 26, 43, 67, 89, 33, 12, 79, 91, 47,
       35, 54, 40, 13, 16, 27, 55, 19, 42, 52, 14,  1, 77, 22, 30, 17, 60,
       41,  8, 42, 62, 73, 28, 4

In [20]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3.element_spec

(TensorSpec(shape=(10,), dtype=tf.int32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(100,), dtype=tf.int32, name=None)))

In [21]:
for a, (b, c) in dataset3:
    print(f"shapes: {a.shape} {b.shape} {c.shape}")

shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)


### Using with generators

In [22]:
def count(stop):
    i = 0
    while i < stop:
        yield i
        i += 1

In [23]:
for i in count(6):
    print(i)

0
1
2
3
4
5


In [24]:
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes=(),)

In [25]:
for batch in ds_counter.batch(10):
    print(batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24]


In [26]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228813984/228813984 [==============================] - 9s 0us/step


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_gen = ImageDataGenerator(rescale=1./255, rotation_range=20)
images, labels = next(img_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [28]:
images.shape

(32, 256, 256, 3)

In [29]:
labels[0]

array([1., 0., 0., 0., 0.], dtype=float32)

In [30]:
dataset = tf.data.Dataset.from_generator(lambda: img_gen.flow_from_directory(flowers), output_types=(tf.float32, tf.float32), 
                                         output_shapes=(images.shape, labels.shape))

In [31]:
for images, labels in dataset.take(1):
    print(images.shape)
    print(labels.shape)

Found 3670 images belonging to 5 classes.
(32, 256, 256, 3)
(32, 5)


In [32]:
x = [1, 2, 3, 4, 5, 6]

dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.repeat(2).batch(5, drop_remainder=True)

In [33]:
for item in dataset:
    print(item.numpy())

[1 2 3 4 5]
[6 1 2 3 4]


## TFRecord Format

In [34]:
with tf.io.TFRecordWriter('my_data.tfrecord') as f:
    f.write(b"This is my first record.")
    f.write(b"This is my second record.")

In [35]:
filepaths = ['my_data.tfrecord']
dataset = tf.data.TFRecordDataset(filepaths)

for item in dataset:
    print(item.numpy())

b'This is my first record.'
b'This is my second record.'


In [36]:
from tensorflow.train import Feature, Example, BytesList, Features, FloatList, Int64List

person_example = Example(features = Features(feature={
    'name': Feature(bytes_list=BytesList(value=[b'Alice'])), 
    'id': Feature(int64_list = Int64List(value=[123])), 
    'emails': Feature(bytes_list = BytesList(value=[b'akoredeadewole8@gmail.com']))
}))

In [37]:
with tf.io.TFRecordWriter('my_contacts.tfrecord') as f:
    f.write(person_example.SerializeToString())

## ExampleGen

### Converting CSV to tf.Example data structure

In [38]:
import os
from tfx import v1 as tfx
from tfx.components import CsvExampleGen

In [39]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [40]:
PIPELINE_NAME = 'AAPL'

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

In [45]:
data_root = '/content/data_root/'

data_path = os.path.join(data_root, 'AAPL.csv')

In [46]:
!head {data_path}

Date,Open,High,Low,Close,Adj Close,Volume
2015-10-29,29.674999,30.172501,29.567499,30.132500,27.667913,204909200
2015-10-30,30.247499,30.305000,29.862499,29.875000,27.431473,197461200
2015-11-02,30.200001,30.340000,29.902500,30.295000,27.817125,128813200
2015-11-03,30.197500,30.872499,30.174999,30.642500,28.136196,182076000
2015-11-04,30.782499,30.955000,30.405001,30.500000,28.005352,179544400
2015-11-05,30.462500,30.672501,30.045000,30.230000,27.876257,158210800
2015-11-06,30.277500,30.452499,30.155001,30.264999,27.908531,132169200
2015-11-09,30.240000,30.452499,30.012501,30.142500,27.795572,135485600
2015-11-10,29.225000,29.517500,29.014999,29.192499,26.919538,236511600


In [47]:
# to run the contextx interactively

context = InteractiveContext()

In [48]:
# create the example gen

example_gen = CsvExampleGen(input_base=data_root)
context.run(example_gen, enable_cache=True)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Loading TFRecord files with ExampleGen

In [49]:
DATA_ROOT = '/content/tfdata'

In [50]:
from tfx.components import ImportExampleGen

example_gen2 = ImportExampleGen(input_base=DATA_ROOT)
context.run(example_gen2)

/usr/local/lib/python3.7/dist-packages/apache_beam/transforms/core.py:1843: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  wrapper = lambda x, *args, **kwargs: [fn(x, *args, **kwargs)]


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Splitting datasets

In [51]:
from tfx.proto import example_gen_pb2

output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
 example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
 example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)]))

In [52]:
example_gen = CsvExampleGen(input_base=data_root, output_config=output)

In [53]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [54]:
for artifact in example_gen.outputs['examples'].get():
 print(artifact)


Artifact(artifact: id: 3
type_id: 14
uri: "/tmp/tfx-interactive-2022-07-16T23_59_13.974616-ynrre88p/CsvExampleGen/examples/3"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\", \"test\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:108268,xor_checksum:1658015919,sum_checksum:1658015919"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.9.0"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "ve

## TensorFlow Data Validation

In [55]:
import tensorflow_data_validation as tfdv

In [56]:
stats = tfdv.generate_statistics_from_csv(data_location='/content/data_root/AAPL.csv', delimiter=',')


# for a TFRecord file it would be
#stats = tfdv.generate_statistics_from_tfrecord(datalocation)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [58]:
type(stats)

tensorflow_metadata.proto.v0.statistics_pb2.DatasetFeatureStatisticsList

In [60]:
# generate schema from the stats
schema = tfdv.infer_schema(stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'Open',FLOAT,required,,-
'High',FLOAT,required,,-
'Low',FLOAT,required,,-
'Close',FLOAT,required,,-
'Adj Close',FLOAT,required,,-
'Volume',INT,required,,-


In [64]:
# compare datasets

train_stats = tfdv.generate_statistics_from_csv(data_location='/content/data_root/AAPL.csv')
val_stats = tfdv.generate_statistics_from_csv(data_location='/content/data_root/GOOG.csv')

In [67]:
tfdv.visualize_statistics(lhs_statistics=train_stats, rhs_statistics=val_stats, lhs_name='TRAIN_SET', rhs_name='VAL_SET')

In [68]:
# checking anomalies
anomalies = tfdv.validate_statistics(statistics=val_stats, schema=schema)

In [69]:
tfdv.display_anomalies(anomalies)

## References

1. https://colab.research.google.com/drive/1kj65CT3d4ninQhJT_NXj5yI5t6vuZ5fa?usp=sharing#scrollTo=kUSUAEqpcV2A
2. https://www.tensorflow.org/tfx/guide/examplegen